# ДЗ к Уроку 1

## Практическое задание по клиентской аналитике 
1. Залить в свою БД данные по продажам (часть таблицы Orders в csv, исходник здесь https://drive.google.com/drive/folders/1C3HqIJcABblKM2tz8vPGiXTFT7MisrML?usp=sharing) 
2. Проанализировать, какой период данных выгружен 
3. Посчитать кол-во строк, кол-во заказов и кол-во уникальных пользователей, кот совершали заказы. 
4. По годам посчитать средний чек, среднее кол-во заказов на пользователя, сделать вывод , как изменялись это показатели Год от года. 
5. Найти кол-во пользователей, кот покупали в одном году и перестали покупать в следующем. 
6. Найти ID самого активного по кол-ву покупок пользователя.

### 1. Залить в свою БД данные по продажам

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('D:/GeekBrains/Database for analytics/orders_20190822.csv', sep=";")

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2002804 entries, 0 to 2002803
Data columns (total 4 columns):
id_o       int64
user_id    int64
price      object
o_date     object
dtypes: int64(2), object(2)
memory usage: 61.1+ MB


In [4]:
df['o_date'] = df['o_date'].astype('datetime64[D]')

In [5]:
df['price'] = [x.replace(',', '.') for x in df['price']]
df['price'] = df['price'].astype(float)

In [6]:
df.describe()

,id_o,user_id,price
count,2.002804e+06,2.002804e+06,2.002804e+06
mean,3.968491e+06,2.381755e+06,2.268164e+03
std,1.682212e+06,1.633211e+06,3.359283e+03
min,1.234491e+06,0.000000e+00,-1.848000e+02
25%,2.460856e+06,1.108048e+06,7.672000e+02
50%,3.978659e+06,2.028087e+06,1.441300e+03
75%,5.391492e+06,3.679133e+06,2.552900e+03
max,6.945534e+06,5.919156e+06,8.190966e+05


### 2. Проанализировать, какой период данных выгружен

In [7]:
df['o_date'].min(), df['o_date'].max()

(Timestamp('2016-01-01 00:00:00'), Timestamp('2017-12-31 00:00:00'))

### 3. Посчитать кол-во строк, кол-во заказов и кол-во уникальных пользователей, кот совершали заказы.

#### Количество строк

In [8]:
len(df)

2002804

#### Количество заказов

In [9]:
len(df["id_o"].unique())

2002804

#### Количество уникальных пользователей, которые совершали заказы

In [10]:
len(df['user_id'].unique())

1015119

### 4. По годам посчитать средний чек, среднее кол-во заказов на пользователя, сделать вывод , как изменялись эти показатели год от года.

#### По годам посчитать средний чек

In [11]:
mean_price = df.groupby(df['o_date'].dt.year)[['price']].mean().rename(columns={'price': 'mean_price'})

In [12]:
mean_price

,mean_price
o_date,
2016,2095.583717
2017,2398.392923


#### По годам посчитать среднее количество заказов на пользователя

In [13]:
mean_order_per_user = {}#pd.DataFrame()
for year in df['o_date'].dt.year.unique():
    df_year = df.loc[(df['o_date'].dt.year == year)]
    mean_order_per_user[year] = df_year.groupby('user_id').count()[['id_o']].mean()[0]
print(mean_order_per_user)

{2016: 1.9352089006317794, 2017: 1.74296603725183}


#### Сделать вывод , как изменялись эти показатели год от года.

Средний чек возрос, однако среднее количество заказов на одного пользователя снизилось. Скорее всего это означает, что пользователи стали делать обращаться в магазин немного реже, но делать более крупные заказы. 

### 5. Найти кол-во пользователей, кот покупали в одном году и перестали покупать в следующем.

In [18]:
years = df['o_date'].dt.year.unique()
df_2016 = df.loc[(df['o_date'].dt.year == years[0])]
df_2017 = df.loc[(df['o_date'].dt.year == years[1])]

In [19]:
len(df_2016[~df_2016.isin(df_2017)].dropna())

861346

### 6. Найти ID самого активного по кол-ву покупок пользователя.

In [20]:
count_orders = df.groupby('user_id').count()[['id_o']].rename(columns={'id_o': 'count_id_o'})#.max()

In [ ]:
#count_orders.max()[0]

In [21]:
most_active_costumers = count_orders.loc[(count_orders['count_id_o'] == count_orders.max()[0])].index[0]

In [22]:
most_active_costumers

765861